In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [8]:
import gensim

In [4]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [5]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [9]:
classes = ["toxic","severe_toxic","obscene","threat","insult","identity_hate"]
y = train[classes].values
tokens = train.comment_text.apply(gensim.utils.simple_preprocess)

In [10]:
tokens.head()

0    [explanation, why, the, edits, made, under, my...
1    [aww, he, matches, this, background, colour, s...
2    [hey, man, really, not, trying, to, edit, war,...
3    [more, can, make, any, real, suggestions, on, ...
4    [you, sir, are, my, hero, any, chance, you, re...
Name: comment_text, dtype: object

In [12]:
phrases = gensim.models.phrases.Phrases(tokens)
tokenizer = gensim.models.phrases.Phraser(phrases)
tokenized_text = list(tokenizer[tokens])

In [13]:
vec = gensim.models.word2vec.Word2Vec(tokenized_text, window=4, size=100, min_count=3, workers=6)

In [14]:
corpus_dict = gensim.corpora.dictionary.Dictionary(tokenized_text)

In [26]:
new_val = [[i+1 for i in corpus_dict.doc2idx(val)] for val in tokenized_text]

In [27]:
print(new_val[1:10])

[[37, 40, 42, 47, 38, 39, 43, 44, 48, 46, 45, 41], [56, 66, 64, 75, 74, 53, 61, 14, 72, 47, 55, 60, 52, 68, 67, 58, 2, 70, 74, 62, 73, 7, 59, 65, 17, 27, 40, 69, 74, 51, 63, 49, 29, 54, 71, 29, 50, 57], [63, 84, 107, 78, 114, 124, 20, 101, 135, 29, 118, 121, 119, 104, 20, 112, 123, 65, 131, 65, 76, 127, 29, 115, 108, 109, 128, 120, 72, 126, 80, 77, 102, 29, 92, 96, 99, 85, 91, 84, 87, 72, 104, 20, 100, 110, 111, 90, 88, 93, 100, 136, 98, 78, 113, 94, 54, 122, 20, 115, 112, 133, 74, 87, 61, 137, 21, 105, 103, 125, 79, 74, 83, 82, 20, 81, 94, 116, 120, 97, 125, 108, 83, 86, 132, 117, 130, 61, 106, 102, 29, 67, 95, 89, 134, 129], [136, 142, 80, 17, 139, 78, 138, 136, 141, 143, 140, 72, 20], [145, 11, 62, 144, 147, 29, 146, 148, 45], [151, 150, 136, 152, 149, 20, 17, 153], [162, 160, 74, 29, 158, 159, 155, 157, 24, 21, 6, 87, 61, 154, 112, 136, 161, 83, 156], [189, 100, 29, 193, 180, 191, 181, 74, 136, 166, 64, 175, 74, 196, 165, 102, 29, 155, 195, 126, 194, 176, 20, 62, 94, 160, 177, 184,

In [28]:
from keras.preprocessing import sequence
train_data = sequence.pad_sequences(new_val,maxlen = 60,truncating = "post",value = 0)

Using TensorFlow backend.
/Users/shashvatkedia/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [33]:
from keras.layers import Convolution1D,MaxPool1D,Dropout,BatchNormalization,Flatten,Dense,Embedding
from keras.models import Sequential
from keras.optimizers import Adam

In [34]:
idx = max(i for j in docs for i in j)
model = Sequential()
model.add(Embedding(idx+1,vec.vector_size,input_length=60))
model.add(Convolution1D(62,4,padding='same'))
model.add(MaxPool1D())
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Convolution1D(124,4,padding='same'))
model.add(MaxPool1D())
model.add(Flatten())
model.add(Dense(6,activation = "sigmoid"))
adam = Adam(lr = 0.001)
model.compile(adam,"binary_crossentropy")

In [ ]:
model.fit(new_val,y,batch_size = 250,epochs = 10,validation_split = 0.2)